In [1]:
import tensorflow as tf
tf.random.set_seed(666)

from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.models import load_model
from tensorflow.keras import optimizers
from tensorflow import keras
import os, shutil
import numpy as np

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

import matplotlib.pyplot as plt

In [2]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.0 MB/s 


In [3]:
import tensorflow_addons as tfa

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data_path = '/content/drive/MyDrive/data/'

train_dir = data_path + 'train'
val_dir = data_path + 'val'
test_dir = data_path + 'test'

In [6]:
img_height = 224
img_width = 224
img_depth = 3
img_size = (img_height, img_width)
batch_size = 50
EPOCHS = 15
AUTO = tf.data.experimental.AUTOTUNE


In [7]:
train_dataset = (
    tf.keras.utils.image_dataset_from_directory(
                  train_dir,
                  seed=123,
                  image_size=(img_height, img_width),
                  batch_size=batch_size)
                  .shuffle(100)
                  .prefetch(AUTO))

val_dataset = (
    tf.keras.utils.image_dataset_from_directory(
                  val_dir,
                  seed=123,
                  image_size=(img_height, img_width),
                  batch_size=batch_size)
                  .shuffle(100)
                  .prefetch(AUTO))

test_dataset = (
    tf.keras.utils.image_dataset_from_directory(
        test_dir,
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size))

Found 5599 files belonging to 8 classes.
Found 1600 files belonging to 8 classes.
Found 800 files belonging to 8 classes.


In [8]:
from keras.preprocessing.image import ImageDataGenerator

batch = 50

train_datagen = ImageDataGenerator()

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    shuffle=True)
label_map = (train_generator.class_indices)

val_generator = test_datagen.flow_from_directory(
        val_dir,
        target_size=(224, 224),
        batch_size=batch,
        class_mode='categorical',
        shuffle=True)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    shuffle=False)

label_map

Found 5599 images belonging to 8 classes.
Found 1600 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


{'Засветы': 0,
 'Малая часть упаковки видна на фото': 1,
 'Не читабилен текст': 2,
 'Нормальные': 3,
 'Размытые': 4,
 'Фото не в том режиме': 5,
 'Фото полки где 4-5 товара и видно что фоткали не основ товар': 6,
 'Фото ценника': 7}

In [9]:
# Teacher model utility
base_model = load_model('/content/drive/MyDrive/dataset/ENetB4new25.h5')


def get_teacher_model():
    inputs = layers.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    classifier = models.Model(inputs=inputs, outputs=x)
    
    return classifier

In [10]:
get_teacher_model().summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 8)                 107598439 
                                                                 
Total params: 107,598,439
Trainable params: 89,924,616
Non-trainable params: 17,673,823
_________________________________________________________________


In [11]:
# Train the teacher model
teacher_model = get_teacher_model()
teacher_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4), metrics=["accuracy"])
# teacher_model.fit(train_generator,
#                   validation_data=val_generator,
#                   epochs=1)

In [12]:
teacher_model.evaluate(test_generator)
teacher_model.save_weights("/content/drive/MyDrive/dataset/teacher_model1.h5")
teacher_model.save_weights("teacher_model1.h5")

16/16 [==============================] - 253s 16s/step - loss: 0.2935 - accuracy: 0.9475


In [13]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = student
        self.student = teacher

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [14]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout,Flatten,\
                                    Dense, Activation, GlobalAveragePooling2D, BatchNormalization,\
                                    AveragePooling2D, Concatenate, Activation
from tensorflow.keras.applications import mobilenet, densenet
from tensorflow.keras.models import load_model, Model, clone_model

In [15]:
student_model = keras.Sequential(
    [
        keras.Input(shape=(224, 224, 3)),
        layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(8),
    ],
    name="student",
)
# Clone student for later comparison
student_scratch = keras.models.clone_model(student_model)

In [16]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller.fit(train_dataset, epochs=3)

# Evaluate student on test dataset
distiller.evaluate(test_dataset)


Epoch 1/3


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


112/112 [==============================] - 882s 522ms/step - sparse_categorical_accuracy: 0.9905 - student_loss: 0.0379 - distillation_loss: 1.9664
Epoch 2/3
112/112 [==============================] - 90s 276ms/step - sparse_categorical_accuracy: 0.9968 - student_loss: 0.0119 - distillation_loss: 1.9672
Epoch 3/3
16/16 [==============================] - 15s 466ms/step - sparse_categorical_accuracy: 0.9513 - student_loss: 0.2911


[0.9512500166893005, 0.09022154659032822]

In [17]:
student_scratch.compile(
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch.fit(train_dataset, epochs=15)


Epoch 1/15
112/112 [==============================] - 64s 39ms/step - loss: 111.1186 - sparse_categorical_accuracy: 0.4460
Epoch 2/15
112/112 [==============================] - 62s 42ms/step - loss: 6.2470 - sparse_categorical_accuracy: 0.8059
Epoch 3/15
112/112 [==============================] - 68s 56ms/step - loss: 1.6853 - sparse_categorical_accuracy: 0.9139
Epoch 4/15
112/112 [==============================] - 62s 42ms/step - loss: 0.7875 - sparse_categorical_accuracy: 0.9557
Epoch 5/15
112/112 [==============================] - 62s 42ms/step - loss: 0.3439 - sparse_categorical_accuracy: 0.9711
Epoch 6/15
112/112 [==============================] - 64s 42ms/step - loss: 0.3021 - sparse_categorical_accuracy: 0.9787
Epoch 7/15
112/112 [==============================] - 62s 42ms/step - loss: 0.1907 - sparse_categorical_accuracy: 0.9852
Epoch 8/15
112/112 [==============================] - 62s 42ms/step - loss: 0.1416 - sparse_categorical_accuracy: 0.9877
Epoch 9/15
112/112 [==========

In [18]:
student_scratch.evaluate(test_dataset)

16/16 [==============================] - 10s 262ms/step - loss: 2.8466 - sparse_categorical_accuracy: 0.9488


[2.8465735912323, 0.9487500190734863]

In [19]:
student_scratch.save_weights("/content/drive/MyDrive/dataset/student_model1.h5")
student_scratch.save_weights("student_model1.h5")

In [20]:
# Investigate the sizes
!ls -lh *.h5

-rw-r--r-- 1 root root 3.1M Nov 19 17:10 student_model1.h5
-rw-r--r-- 1 root root 411M Nov 19 16:33 teacher_model1.h5


In [21]:
teacher_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 8)                 107598439 
                                                                 
Total params: 107,598,439
Trainable params: 89,924,616
Non-trainable params: 17,673,823
_________________________________________________________________


In [22]:
student_model.summary()

Model: "student"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 16)      448       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 112, 112, 16)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 32)        4640      
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 8)                 802824    
                                                           

In [23]:
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(train_dataset).batch(1).take(100):
        yield [input_value]

def convert_to_tflite(model, tflite_file):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_data_gen
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8
    tflite_quant_model = converter.convert()

    open(tflite_file, 'wb').write(tflite_quant_model)

In [ ]:
convert_to_tflite(teacher_model, "teacher.tflite")
convert_to_tflite(student_model, "student.tflite")

In [ ]:
!ls -lh *.tflite

In [ ]:
teacher_model.save_weights("/content/drive/MyDrive/dataset/teacher.tflite")
student_scratch.save_weights("/content/drive/MyDrive/dataset/student.tflite")